In [1]:
%load_ext tensorboard


In [2]:
%%capture
!pip install datasets
!pip install tokenizers
!pip install torchmetrics


In [3]:
!git clone https://github.com/PranavInani/transformer-from-scratch


Cloning into 'transformer-from-scratch'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 58 (delta 28), reused 43 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 25.94 KiB | 5.19 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [2]:
%%capture
%cd /kaggle/working/transformer-from-scratch

In [3]:
%tensorboard --logdir runs

<IPython.core.display.Javascript object>

In [4]:
from config import get_config

# Update configuration for Colab environment
cfg = get_config()
cfg['model_folder'] = 'weights'  # Save weights in the working directory
cfg['batch_size'] = 64  # Increased batch size to 64
cfg['num_epochs'] = 3  # Reduced number of epochs for testing
cfg['preload'] = 'latest'  # Start training from scratch (or use 'latest' if you have previous weights)
cfg['use_amp'] = False 
cfg['seq_len'] = 95
# Enable mixed precision training for faster execution

from train import train_model

# Train the model with the updated configuration
train_model(cfg)

Using device: cuda
Device name: Tesla P100-PCIE-16GB
Device memory: 15.887939453125 GB


README.md:   0%|          | 0.00/3.14k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/85.7k [00:00<?, ?B/s]

...

/kaggle/working/transformer-from-scratch/train.py:270: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()
/kaggle/working/transformer-from-scratch/train.py:280: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case wh

Preloading model cfilt/iitb-english-hindi_weights/tmodel_best.pt


Processing Epoch 02: 100%|██████████| 25924/25924 [3:06:48<00:00,  2.31it/s, loss=3.500]  


--------------------------------------------------------------------------------
    SOURCE: The DSP of the Roop-nagar vigilance bureau, Balveer Singh said, if the general public were vigilant against corruption then a corruption free society could be built.
    TARGET: रूपनगर विजिलेंस ब्यूरो के डीएसपी बलवीर सिंह ने यदि आमलोग भ्रष्टाचार के खिलाफ विजिलेंस का साथ दे तो भ्रष्टाचारमुक्त समाज का निर्माण किया जा सकता है।
 PREDICTED: रो प् - नगर चौक सी ब्यूरो के डी पी एस ने कहा , यदि आम जनता भ्रष्टाचार के खिलाफ सावधान है तो भ्रष्टाचार मुक्त समाज का निर्माण किया जा सकता है ।
--------------------------------------------------------------------------------
    SOURCE: Through this state-wide signature campaign, 10 Lakh (1 million)signatures across the state will be collected and handed over to the Governor.
    TARGET: इस राज्यव्यापी हस्ताक्षर अभियान के माध्यम से प्रदेश भर में 10 लाख हस्ताक्षर करवाकर राज्यपाल को सौंपे जाएंगे।
 PREDICTED: इस राज्य व्यापी हस्ताक्षर अभियान के माध्यम से , 10 लाख ( 1

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `CharErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CharErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `WordErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `WordErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `BLEUScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `BLEUScore` from `torchmetrics.text` instead.
  _future_warning(


# Inference

In [5]:
from pathlib import Path
import torch
import torch.nn as nn
from config import get_config, latest_weights_file_path
from train import get_model, get_ds, run_validation
from translate import translate


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
config = get_config()
train_dataloader, val_dataloader, test_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename = latest_weights_file_path(config)
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])


Using device: cuda
Max length of source sentence: 3015
Max length of target sentence: 2763


<ipython-input-6-f2218c07bbfe>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(model_filename)


<All keys matched successfully>

In [7]:
run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: print(msg), 0, None, num_examples=15)

--------------------------------------------------------------------------------
    SOURCE: Ask yourself whether those orders were followed in the states?
    TARGET: अपने आप से पूछिए कि क्या उन आदेशों का राज्यों में पालन हुआ?
 PREDICTED: क्या राज्य में इन आदेशों का पालन किया गया है ?
--------------------------------------------------------------------------------
    SOURCE: The Corporate Services Deputy V.P. was in the city but could not meet the Union officers.
    TARGET: डिप्टी वीपी कॉरपोरेट सर्विसेज शहर में मौजूद रहे लेकिन यूनियन पदाधिकारियों से बातचीत नहीं हो सकी।
 PREDICTED: कंपनी सेवा उप वी . पी . शहर में थी , लेकिन संघ के अधिकारियों से मिलने नहीं पा सका ।
--------------------------------------------------------------------------------
    SOURCE: On Thursday intensive checks of trucks were carried out.
    TARGET: गुरुवार को ट्रकों की सघन जाच हो रही थी।
 PREDICTED: बृह स्पति वार के गहन परीक्षण ों को किया गया ।
-----------------------------------------------------------------

In [8]:
t = translate("How is your mood now?")

Using device: cuda


/kaggle/working/transformer-from-scratch/translate.py:102: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(model_filename, map_location=device)


    SOURCE: How is your mood now?
 PREDICTED: अब तुम्हारा मन कैसा है ?


In [9]:
%run test.py

Using device: cuda
Loading model from cfilt/iitb-english-hindi_weights/tmodel_best.pt


/kaggle/working/transformer-from-scratch/test.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(model_filename, map_location=device)


Starting evaluation on test dataset...


Testing: 100%|██████████| 2507/2507 [09:50<00:00,  4.24it/s]



Test Results:
Character Error Rate (CER): 0.5964
Word Error Rate (WER): 0.9610
BLEU Score: 0.0000


/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:70: FutureWarning: Importing `char_error_rate` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `char_error_rate` from `torchmetrics.text` instead.
  _future_warning(



Best Translation Example:
Source: And I think about my father.
Target: और मैं अपने पिता के बारे में सोचता हूँ।
Predicted: और मैं अपने पिता के बारे में सोचता हूँ ।
CER: 0.0256

Worst Translation Example:
Source: Frontier's new carry-on fee won't start until summer, though a date hasn't been set.
Target: Frontier का नया कैरी - ऑन फीस, गर्मियों तक शुरू नहीं  होगा यद्यपि एक तिथि निर्धारित नहीं  किया गया है.
Predicted: सीमा पार करने वाले के नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए - नए
CER: 2.0980

Analysis plot saved as 'length_vs_error.png'
Detailed results saved to 'test_results.txt'


In [41]:
from config import get_config
get_config()

{'batch_size': 8,
 'num_epochs': 20,
 'lr': 0.0001,
 'seq_len': 70,
 'd_model': 512,
 'datasource': 'cfilt/iitb-english-hindi',
 'lang_src': 'en',
 'lang_tgt': 'hi',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': 'latest',
 'tokenizer_file': 'tokenizer_{0}_bpe.json',
 'experiment_name': 'runs/tmodel',
 'vocab_size': 16000,
 'use_amp': True}

In [46]:
import config

In [45]:
config.get_config()

{'batch_size': 8,
 'num_epochs': 20,
 'lr': 0.0001,
 'seq_len': 70,
 'd_model': 512,
 'datasource': 'cfilt/iitb-english-hindi',
 'lang_src': 'en',
 'lang_tgt': 'hi',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': 'latest',
 'tokenizer_file': 'tokenizer_{0}_bpe.json',
 'experiment_name': 'runs/tmodel',
 'vocab_size': 16000,
 'use_amp': True}